In [1]:
# -*- coding: utf-8 -*-
import asyncio
import websockets
import json
import logging
import base64
import time
import sys
from typing import Optional, Set

try:
    # Attempt to import JetBot libraries
    from jetbot import Robot, Camera, bgr8_to_jpeg
    JETBOT_AVAILABLE = True
except ImportError as e:
    print(f"Warning: Could not import jetbot library: {e}. Hardware functions disabled.")
    # Define placeholders if import fails
    Robot = None
    Camera = None
    bgr8_to_jpeg = None
    JETBOT_AVAILABLE = False

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("JetBotDriver")

# --- Configuration ---
WEBSOCKET_HOST = "0.0.0.0"
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300
FPS = 1.0 # Target FPS for sending frames
FRAME_SEND_INTERVAL = 1.0 / FPS

# --- Global State ---
robot: Optional[Robot] = None
camera: Optional[Camera] = None
hardware_initialized = False

# --- Hardware Initialization ---
if JETBOT_AVAILABLE:
    try:
        robot = Robot()
        try:
            camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
        except TypeError:
            logger.warning("Camera.instance() failed, trying legacy Camera().")
            camera = Camera(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

        if robot is None or camera is None:
             raise RuntimeError("Failed to create Robot or Camera object.")

        logger.info("JetBot hardware initialized.")
        time.sleep(1.5) # Allow camera warmup
        if camera.value is None:
            logger.warning("Camera init OK, but initial frame is None. Check hardware.")
        hardware_initialized = True

    except FileNotFoundError:
        logger.error("JetBot init failed: Motor driver files not found. Check I2C setup.")
    except RuntimeError as e:
         logger.error(f"JetBot hardware init failed: {e}", exc_info=False) # Keep log cleaner
    except Exception as e:
        logger.error(f"General JetBot hardware initialization failed: {e}", exc_info=True)
else:
    logger.warning("JetBot library not found. Running without hardware support.")


# --- Command Handling (Direct Speed Control) ---
async def handle_speed_command(speed_data: dict):
    """Sets motor speeds based on received JSON data."""
    if not hardware_initialized or not robot: return

    try:
        left_speed = float(speed_data.get('left_speed', 0.0))
        right_speed = float(speed_data.get('right_speed', 0.0))

        # Clamp speeds to the valid range [-1.0, 1.0]
        left_speed = max(-1.0, min(1.0, left_speed))
        right_speed = max(-1.0, min(1.0, right_speed))

        robot.set_motors(left_speed, right_speed)
        # logger.debug(f"Motors set: L={left_speed:.2f}, R={right_speed:.2f}")

    except (TypeError, ValueError) as e:
        logger.warning(f"Invalid speed data format: {speed_data}. Error: {e}. Stopping.")
        if robot: try: robot.stop()
        except Exception as se: logger.error(f"Failed stop after invalid data: {se}")
    except Exception as e:
        logger.error(f"Error executing speed command: {e}")
        if robot: try: robot.stop()
        except Exception as se: logger.error(f"Failed stop after execution error: {se}")

# --- Camera Frame Sending ---
async def send_camera_frames(websocket: websockets.WebSocketServerProtocol):
    """Continuously sends camera frames to the client."""
    if not hardware_initialized or not camera or not bgr8_to_jpeg:
        logger.warning("Camera/Conversion unavailable, cannot send frames.")
        return

    loop = asyncio.get_event_loop()
    last_frame_time = loop.time()

    while websocket.open:
        current_time = loop.time()
        wait_time = FRAME_SEND_INTERVAL - (current_time - last_frame_time)
        if wait_time > 0: await asyncio.sleep(wait_time)
        last_frame_time = loop.time()

        try:
            frame = camera.value
            if frame is None: await asyncio.sleep(0.1); continue
            if not bgr8_to_jpeg: logger.error("bgr8_to_jpeg unavailable!"); break

            jpeg_data = bgr8_to_jpeg(frame)
            image_base64 = base64.b64encode(jpeg_data).decode('utf-8')
            # Send only if connection is still open after encoding
            if websocket.open:
                await websocket.send(json.dumps({"image": image_base64}))

        except (websockets.exceptions.ConnectionClosed, websockets.exceptions.WebSocketException):
            # logger.debug("Connection closed during frame send.") # Reduce noise
            break
        except Exception as e:
            logger.error(f"Frame send/process error: {e}", exc_info=True)
            await asyncio.sleep(0.5) # Wait after error before retrying

# --- WebSocket Connection Handler ---
async def websocket_handler(websocket: websockets.WebSocketServerProtocol, path: str):
    """Handles a single client connection."""
    client_ip = websocket.remote_address[0]
    logger.info(f"Client connected: {client_ip}")
    frame_sender_task = None

    # Start sending frames only if camera is available
    if hardware_initialized and camera:
        frame_sender_task = asyncio.ensure_future(send_camera_frames(websocket))
    else:
        logger.warning(f"Camera not available for client {client_ip}")

    try:
        # Listen for incoming speed commands
        async for message in websocket:
            try:
                data = json.loads(message)
                if 'left_speed' in data and 'right_speed' in data:
                    await handle_speed_command(data)
            except json.JSONDecodeError:
                logger.warning(f"Invalid JSON from {client_ip}: {message[:100]}...")
            except Exception as e:
                logger.error(f"Msg handle error from {client_ip}: {e}", exc_info=True)

    except websockets.exceptions.ConnectionClosed:
        logger.info(f"Connection from {client_ip} closed.")
    except Exception as e:
        logger.error(f"Handler error for {client_ip}: {e}", exc_info=True)
    finally:
        logger.info(f"Disconnecting client: {client_ip}")
        if frame_sender_task and not frame_sender_task.done():
            frame_sender_task.cancel()
        # Stop motors on disconnect for safety
        if hardware_initialized and robot:
            logger.info("Client disconnected, stopping motors.")
            try: robot.stop()
            except Exception as e: logger.error(f"Failed stop on disconnect: {e}")

# --- Main Function (Simplified Loop) ---
async def main():
    """Starts the WebSocket server and keeps it running."""
    if hardware_initialized and robot:
        try: robot.stop() # Initial stop
        except Exception as e: logger.warning(f"Could not stop motors initially: {e}")

    while True:
        try:
            server = await websockets.serve(
                websocket_handler,
                WEBSOCKET_HOST,
                WEBSOCKET_PORT,
                ping_interval=20,
                ping_timeout=20
            )
            logger.info(f"WebSocket server running on ws://{WEBSOCKET_HOST}:{WEBSOCKET_PORT}")
            # Keep the main() coroutine running indefinitely
            await asyncio.Future() # This waits forever
        except OSError as e:
             if "Address already in use" in str(e):
                  logger.error(f"Port {WEBSOCKET_PORT} is already in use. Retrying...")
             else: logger.error(f"Server OS error: {e}")
             await asyncio.sleep(10)
        except Exception as e:
            logger.error(f"Server error: {e}", exc_info=True)
            logger.info("Retrying server start in 5 seconds...")
            await asyncio.sleep(5)


# --- Main Execution Block (Simplified for Python 3.6) ---
if __name__ == "__main__":
    logger.info("Starting JetBot Driver...")
    loop = asyncio.get_event_loop()

    try:
        # Attempt to run the main server coroutine until it completes or is interrupted
        loop.run_until_complete(main())

    except KeyboardInterrupt:
        logger.info("KeyboardInterrupt received. Stopping...")
        # Cleanup is handled in the finally block

    except RuntimeError as e:
        # Handle the "event loop is already running" error specifically
        if "already running" in str(e):
             logger.warning("Event loop was already running (Jupyter?). Server task might be running in background.")
             logger.warning("Use Kernel->Interrupt or stop the cell to exit.")
             # Keep the script alive so the background task can run
             try:
                  while True: time.sleep(60) # Sleep for long intervals
             except KeyboardInterrupt:
                   logger.info("KeyboardInterrupt received while waiting.")
        else:
             # Re-raise other RuntimeErrors
             logger.error(f"Unhandled Runtime Error: {e}", exc_info=True)
             raise e
    except Exception as e:
        # Catch any other unexpected errors during startup
        logger.error(f"Unhandled exception in main execution block: {e}", exc_info=True)

    finally:
        logger.info("Initiating final cleanup...")

        # 1. Stop Motors (Best Effort)
        if hardware_initialized and robot:
            try:
                logger.info("Stopping motors during final cleanup.")
                robot.stop()
            except Exception as e:
                logger.error(f"Error stopping motors during cleanup: {e}")

        # 2. Cancel All Running Tasks (Best Effort for Python 3.6)
        if loop and not loop.is_closed():
            try:
                tasks = [t for t in asyncio.Task.all_tasks(loop=loop) if not t.done()]
                if tasks:
                    logger.info(f"Cancelling {len(tasks)} pending tasks...")
                    for task in tasks:
                        task.cancel()
                    # Wait for cancellations - Use run_until_complete if loop is stopped
                    # This part is tricky if the loop was stopped externally (like Jupyter interrupt)
                    try:
                         # Only run if loop isn't already closed, might still fail if stopped externally
                         if not loop.is_closed():
                              loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))
                         else:
                              logger.warning("Loop closed before tasks could be gathered.")
                    except RuntimeError as gather_err: # Catch if run_u_c is called on stopped loop
                         logger.warning(f"Could not gather cancelled tasks (loop likely stopped): {gather_err}")
                    except Exception as gather_err:
                         logger.error(f"Error gathering cancelled tasks: {gather_err}")
                else:
                    logger.info("No pending tasks to cancel.")

                # 3. Shutdown Async Generators (Python 3.6+)
                logger.info("Shutting down async generators...")
                try:
                    if not loop.is_closed(): # Check again before running
                        loop.run_until_complete(loop.shutdown_asyncgens())
                    else:
                         logger.warning("Loop closed before asyncgen shutdown.")
                except RuntimeError as gen_err:
                    logger.warning(f"Error shutting down asyncgens (may be expected): {gen_err}")
                except Exception as gen_err:
                    logger.error(f"Unexpected error during asyncgen shutdown: {gen_err}")

                # 4. Close the Loop
                logger.info("Closing event loop.")
                loop.close()
                logger.info("Event loop closed.")

            except Exception as final_cleanup_err:
                 logger.error(f"Error during final loop cleanup: {final_cleanup_err}")
        elif loop and loop.is_closed():
             logger.info("Event loop was already closed.")
        else:
            logger.warning("No active event loop found for final cleanup.")

        logger.info("JetBot Driver application finished.")

SyntaxError: invalid syntax (<ipython-input-1-be9b07262b60>, line 86)